In [70]:
# https://jessesw.com/XG-Boost/
import numpy as np
import pandas as pd

def describe_categorical(df):
    from IPython.display import display, HTML
    display(HTML(df[df.columns[df.dtypes == 'object']].describe().to_html()))
    
def unique_col_values(df):
    l = []
    index = df.columns
    columns = ['Uniques', 'DT', 'Nulls (%)']
    for column in index:
        u = len(df[column].unique())
        c = len(df[column])
        n = df[column].isnull().sum()
        l.append([u, df[column].dtype, n/c*100])
        
    return pd.DataFrame(l,index=index,columns=columns).sort_values(by='Nulls (%)', ascending=False)    

In [7]:
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',
                      skiprows = 1, header = None) # Make sure to skip a row for the test set

In [19]:
print(train_set.shape)
train_set.head()

(32561, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [20]:
print(test_set.shape)
test_set.head()

(16281, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [12]:
describe_categorical(train_set)
describe_categorical(test_set)

,1,3,5,6,7,8,9,13,14
count,32561,32561,32561,32561,32561,32561,32561,32561,32561
unique,9,16,7,15,6,5,2,42,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22696,10501,14976,4140,13193,27816,21790,29170,24720


,1,3,5,6,7,8,9,13,14
count,16281,16281,16281,16281,16281,16281,16281,16281,16281
unique,9,16,7,15,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K.
freq,11210,5283,7403,2032,6523,13946,10860,14662,12435


- We don’t have a column header for our data
- There seem to be some unknown values in the fifth row of the test set (the question marks) we need to deal with
- The target values have periods at the end in the test set but do not in the training set (<=50K. vs. <=50K)

In [14]:
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

Next, let’s check to see if we can identify any of these missing values.

In [17]:
unique_col_values(train_set)

,Uniques,DT,Nulls (%)
age,73,int64,0.0
workclass,9,object,0.0
fnlwgt,21648,int64,0.0
education,16,object,0.0
education_num,16,int64,0.0
marital_status,7,object,0.0
occupation,15,object,0.0
relationship,6,object,0.0
race,5,object,0.0
sex,2,object,0.0


In [18]:
unique_col_values(test_set)

,Uniques,DT,Nulls (%)
age,73,int64,0.0
workclass,9,object,0.0
fnlwgt,12787,int64,0.0
education,16,object,0.0
education_num,16,int64,0.0
marital_status,7,object,0.0
occupation,15,object,0.0
relationship,6,object,0.0
race,5,object,0.0
sex,2,object,0.0


In [26]:
train_set['occupation'].value_counts(dropna=False)

 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial      4066
 Adm-clerical         3770
 Sales                3650
 Other-service        3295
 Machine-op-inspct    2002
 ?                    1843
 Transport-moving     1597
 Handlers-cleaners    1370
 Farming-fishing       994
 Tech-support          928
 Protective-serv       649
 Priv-house-serv       149
 Armed-Forces            9
Name: occupation, dtype: int64

In [46]:
train_set[train_set.index == 20068]['workclass'].values[0]

' ?'

It turns out the question mark was actually entered with a space first. Let’s do a simple test to see what would happen if we dropped all rows that contain an unknown marked with a ‘ ?’.

In [52]:
print(train_set.replace(' ?', np.nan).dropna().shape)
print(test_set.replace(' ?', np.nan).dropna().shape)

(30162, 15)
(15060, 15)


These must be our missing rows since the numbers add up now if we drop them. Let’s apply this change to our test and training sets.

In [55]:
train_nomissing = train_set.replace(' ?', np.nan).dropna()
test_nomissing = test_set.replace(' ?', np.nan).dropna()

In [57]:
test_nomissing['wage_class'].value_counts()

 <=50K.    11360
 >50K.      3700
Name: wage_class, dtype: int64

Now that we have taken care of the missing value problem, we still have an issue with the target income thresholds being encoded slightly differently in test vs. train. We need these to match up appropriately, so we are going to need to fix either the test or training set to make them match up. Let’s replace all of the ‘<=50K.’ with ‘<=50K’ and the same for ‘>50K.’ with ‘>50K’, so essentially, we are just dropping the periods. This is also encoded with a space so include this in the string. We can use the replace method from pandas to fix this.

In [58]:
test_nomissing['wage_class'] = test_nomissing['wage_class'].replace({' <=50K.': ' <=50K',
                                                                     ' >50K.' : ' >50K'})

In [59]:
test_nomissing['wage_class'].value_counts()

 <=50K    11360
 >50K      3700
Name: wage_class, dtype: int64

In [60]:
train_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

Now these look the same. There is one thing we need to do, however, before applying XGBoost. We need to make sure everything encoded as a string is turned into a variable.

### Applying Ordinal Encoding to Categoricals



In [61]:
combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) # Stacks them vertically

In [66]:
combined_set['e2'] = pd.Categorical(combined_set['education']).codes

In [68]:
combined_set['education'].value_counts()

 HS-grad         14783
 Some-college     9899
 Bachelors        7570
 Masters          2514
 Assoc-voc        1959
 11th             1619
 Assoc-acdm       1507
 10th             1223
 7th-8th           823
 Prof-school       785
 9th               676
 12th              577
 Doctorate         544
 5th-6th           449
 1st-4th           222
 Preschool          72
Name: education, dtype: int64

In [69]:
combined_set['e2'].value_counts()

11    14783
15     9899
9      7570
12     2514
8      1959
1      1619
7      1507
0      1223
5       823
14      785
6       676
2       577
10      544
4       449
3       222
13       72
Name: e2, dtype: int64